# Mutation Free Energy Calculations using BioExcel Building Blocks (biobb)
### Based on the official pmx tutorial: http://pmx.mpibpc.mpg.de/sardinia2018_tutorial1/index.html
***
This tutorial aims to illustrate how to compute a **fast-growth** **mutation free energy** calculation, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Staphylococcal nuclease** protein (PDB code 1STN), a small, minimal protein, appropriate for a short tutorial. 

The **non-equilibrium free energy calculation** protocol performs a **fast alchemical transition** in the direction **WT->Mut** and back **Mut->WT**. The two **equilibrium trajectories** needed for the tutorial, one for **Wild Type (WT)** and another for the **Mutated (Mut)** protein (Isoleucine 10 to Alanine -I10A-), have already been generated and are included in this example.  We will name **WT as stateA** and **Mut as stateB**.

<img src="schema.png" />

The tutorial calculates the **free energy difference** in the folded state of a protein. Starting from **two 1ns-length independent equilibrium simulations** (WT and mutant), snapshots are selected to start **fast (50ps) transitions** driving the system in the **forward** (WT to mutant) and **reverse** (mutant to WT) directions, and the **work values** required to perform these transitions are collected. With these values, **Crooks Gaussian Intersection** (CGI), **Bennett Acceptance Ratio** (BAR) and **Jarzynski estimator** methods are used to calculate the **free energy difference** between the two states.

*Please note that for the sake of disk space this tutorial is using 1ns-length equilibrium trajectories, whereas in the [original example](http://pmx.mpibpc.mpg.de/sardinia2018_tutorial1/eq.mdp) the equilibrium trajectories used were obtained from 10ns-length simulations.*
***
**Biobb modules** used:

 - [biobb_pmx](https://github.com/bioexcel/biobb_pmx): Tools to setup and run Alchemical Free Energy calculations.
 - [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [os](https://docs.python.org/3/library/os.html): Python miscellaneous operating system interfaces
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.

### Conda Installation and Launch

```console
 git clone https://github.com/bioexcel/biobb_wf_pmx_tutorial.git
 cd biobb_wf_pmx_tutorial
 conda env create -f conda_env/environment.yml
 conda activate biobb_wf_pmx_tutorial
 conda install -y -c bioconda biobb_analysis==2.0.1
 jupyter-notebook biobb_wf_pmx_tutorial/notebooks/biobb_wf_pmx_tutorial.ipynb
  ``` 

***
### Pipeline steps:
 1. [Workflow required files](#input)
 2. [Extract Snapshots from Equilibrium Trajectories](#extract)
 3. [Modelling mutated structure](#mut)
 4. [Create Protein System Topology](#top)
 5. [Generate Hybrid Topology](#hybridtop)
 6. [Creating an index file for the dummy atoms in the morphed structure](#index)
 7. [Energetically Minimize the System](#min)
 8. [Equilibrate the System (NPT)](#npt)
 9. [Free Energy Simulation](#free)
 10. [Free Energy Estimation](#post)
 11. [Output Files](#output)
 12. [Questions & Comments](#questions)

***
<img style="width:400px;" src="logo.png" />

***

<a id="input"></a>
## Workflow required files
Workflow **Input files** needed:
 - **stateA_traj**: Equilibrium trajectory for the **WT protein**.
 - **stateB_traj**: Equilibrium trajectory for the **Mutated protein**.
 - **stateA_tpr**: **WT protein topology** (GROMACS tpr format).
 - **stateB_tpr**: **Mutated protein topology** (GROMACS tpr format).
 
Auxiliar **force field libraries** needed:
 - **mutff45** (folder): pmx mutation force field libraries. 
 
Collected **transitions work values**:  
 - **dhdlA.zip**: **Forward** work values required to illustrate how to perform the last step of the pmx workflow with real values, to extract the **free energy**.
 - **dhdlB.zip**: **Reverse** work values required to illustrate how to perform the last step of the pmx workflow with real values, to extract the **free energy**.
 

In [1]:
import os
import zipfile

cwd = os.getcwd()
gmxlib = cwd + "/pmx_tutorial/mutff45"

stateA_traj = cwd + "/pmx_tutorial/stateA_1ns.xtc"
stateA_tpr = cwd + "/pmx_tutorial/stateA.tpr"

stateB_traj = cwd + "/pmx_tutorial/stateB_1ns.xtc"
stateB_tpr = cwd + "/pmx_tutorial/stateB.tpr"

<a id="extract"></a>
## Extract Snapshots from Equilibrium Trajectories
**Extract snapshots** from **equilibrium trajectories** (stateA, stateB).

In the **fast-growth method**, the **free energy difference** in the folded state of a protein is calculated from **two independent equilibrium simulations**, **WT** and **mutant**. These simulations need to **sufficiently sample** the **end state ensembles**, as the **free energy accuracy** will depend on the **sampling convergence**. Typically the equilibrium simulations are in the **nanosecond to microsecond time range**. From the generated trajectories, a suitable number of **snapshots** are selected to start **fast (typically 10 - 200 ps) transitions** driving the system in the **forward (WT to mutant)** and **reverse (mutant to WT)** directions. This particular step **extracts the snapshots** from the **input equilibrium trajectories**. 

In this tutorial, just **5 snapshots** for each state (forward, reverse) are generated, for **illustration purposes**. In a real example, both **input trajectories should be longer** (10ns upwards) and the **number of extracted snapshots higher**. The number of suitable number of snapshots depend on the **complexity of the mutation**. Larger perturbations will take **longer to converge** in the simulation, so a smaller or more conservative change might be more promising. 

**WT snapshots**: **stateA**   //    **Mutant snapshots**: **stateB**
***
**Building Blocks** used:
 - [GMXTrjConvStrEns](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#gromacs-gmx-trjconv-str-ens-module) from **biobb_analysis.gromacs.gmx_trjconv_str_ens**
***

In [2]:
# GMXTrjConvStrEns: extract an ensemble of snapshots from a GROMACS trajectory file
# Import module
from biobb_analysis.gromacs.gmx_trjconv_str_ens import GMXTrjConvStrEns

#### State A ####

# Create prop dict and inputs/outputs (StateA)                          
output_framesA = 'stateA_frames.zip'

prop = {
    'selection' : 'System',
    'start': 1,    # To be changed to generate as many snapshots as needed
    'end': 1000,   # To be changed to generate as many snapshots as needed
    'dt': 200,     # To be changed to generate as many snapshots as needed
    'output_name': 'frameA',
    'output_type': 'pdb'
}

# Create and launch bb (StateA)        
GMXTrjConvStrEns(input_traj_path=stateA_traj,
                 input_top_path=stateA_tpr,
                 output_str_ens_path=output_framesA,
                 properties=prop).launch()

# Extract stateA (WT) frames
with zipfile.ZipFile(output_framesA, 'r') as zip_f:
    zip_f.extractall()
    stateA_pdb_list = zip_f.namelist()
    
#### State B #### 
    
# Create prop dict and inputs/outputs (StateB)                                
output_framesB = 'stateB_frames.zip'  
    
prop = {
    'selection' : 'System',
    'start': 1,   # To be changed to generate as many snapshots as needed
    'end': 1000,  # To be changed to generate as many snapshots as needed
    'dt': 200,    # To be changed to generate as many snapshots as needed
    'output_name': 'frameB',
    'output_type': 'pdb'
}

# Create and launch bb (StateB)
GMXTrjConvStrEns(input_traj_path=stateB_traj,
                 input_top_path=stateB_tpr,
                 output_str_ens_path=output_framesB,
                 properties=prop).launch()
    
# # Extract stateB (Mutant) frames
with zipfile.ZipFile(output_framesB, 'r') as zip_f:
    zip_f.extractall()
    stateB_pdb_list = zip_f.namelist()

2019-12-04 17:44:42,387 [MainThread  ] [INFO ]  Creating 940f0722-b2f3-467a-b6c4-e5383e27f18d temporary folder
2019-12-04 17:44:42,388 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:44:44,236 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/stateA_1ns.xtc -s /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/stateA.tpr -b 1 -e 1000 -dt 200 -sep -o 940f0722-b2f3-467a-b6c4-e5383e27f18d/frameA.pdb

2019-12-04 17:44:44,237 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:44:44,238 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2019-12-04 17:44:44,238 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:

2019-12-04 17:44:46,243 [MainThread  ] [INFO ]  Adding:
2019-12-04 17:44:46,244 [MainThread  ] [INFO ]  [PosixPath('0b1fc312-6c44-44c1-9efa-f55168a2898f/frameB0.pdb'), PosixPath('0b1fc312-6c44-44c1-9efa-f55168a2898f/frameB1.pdb'), PosixPath('0b1fc312-6c44-44c1-9efa-f55168a2898f/frameB2.pdb'), PosixPath('0b1fc312-6c44-44c1-9efa-f55168a2898f/frameB3.pdb'), PosixPath('0b1fc312-6c44-44c1-9efa-f55168a2898f/frameB4.pdb')]
2019-12-04 17:44:46,247 [MainThread  ] [INFO ]  to: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/stateB_frames.zip
2019-12-04 17:44:46,256 [MainThread  ] [INFO ]  Removed temporary folder: 0b1fc312-6c44-44c1-9efa-f55168a2898f


***

## WARNING
For the sake of **time**, this tutorial will only compute the work values for **ONE PARTICULAR frame** of each state. In order to reproduce a **real calculation**, the steps presented in this notebook should be **repeated for the rest of the frames**. The **ensemble of computed work values** should then be used in the **final step of the workflow**, the [free energy calculation](#post).    
***

In [3]:
# Prepare Mutation Free Energy calculation for ONE PARTICULAR frame of each state
# (to be repeated for the rest of the frames)
pdbA = stateA_pdb_list[0]
pdbB = stateB_pdb_list[0]

<a id="mut"></a>
## Modelling mutated structures
**Modelling mutated structures** with the desired new residues using **pmx** package. In this case:

**Isoleucine** residue nº10 will be mutated to an **Alanine** in the **forward** transition.

**Alanine** residue nº10 will be mutated to an **Isoleucine** in the **reverse** transition.

*Note:* This tutorial is using a **Docker container** to run **pmx**. To use a local installation of **pmx**, just modify the appropriate properties parameter **pmx_cli_path**. 
***
**Building Blocks** used:
 - [Mutate](https://biobb-pmx.readthedocs.io/en/latest/pmx.html#module-pmx.mutate) from **biobb_pmx.pmx.mutate**
***

## WARNING
The first time that **pmx Docker** container is used, it will be **automatically downloaded** and integrated in the Docker system. This can take **some time, in the order of minutes**, depending on the available **network**. Please, be patient if the workflow gets stuck in this step. 

This process is **only done once**. When the docker is downloaded, the system will locate it and **it will not be required to download again**.     
***

In [4]:
# pmx mutate: Mutate command from pmx package

# Import module
from biobb_pmx.pmx.mutate import Mutate

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_structure_mutA = 'mutA.pdb'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'mutation_list' : 'Ile10Ala',
    'container_path': 'docker',
    'container_image' : 'mmbirb/pmx'
    #'pmx_cli_path' : 'PATH/pmx-master/pmx/scripts/cli.py'
}
# Create and launch bb
Mutate(input_structure_path=pdbA,
       output_structure_path=output_structure_mutA,
       properties=prop).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_structure_mutB = 'mutB.pdb'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'mutation_list' : 'Ala10Ile',
    'container_path': 'docker',
    'container_image' : 'mmbirb/pmx'
    #'pmx_cli_path' : 'PATH/pmx-master/pmx/scripts/cli.py'
}
# Create and launch bb
Mutate(input_structure_path=pdbB,
       output_structure_path=output_structure_mutB,
       properties=prop).launch()


2019-12-04 17:44:54,340 [MainThread  ] [INFO ]  Using Docker image mmbirb/pmx
2019-12-04 17:45:00,522 [MainThread  ] [INFO ]  docker run -v /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/429be53c-8ce0-427e-ae6c-3540892fe26a:/data mmbirb/pmx /bin/bash -c "cli.py mutate -f /data/frameA0.pdb -o /data/mutA.pdb -ff amber99sb-star-ildn-mut -script /data/mutations.txt"

2019-12-04 17:45:00,526 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:45:00,528 [MainThread  ] [INFO ]  ---------------------------------------------------------------------------------------------------------
 Program: mutate | pmx version 2.0+9.geb3590f
---------------------------------------------------------------------------------------------------------
     File Options          | Type(s)|Mode    | File(s)                | Description 
---------------------------------------------------------------------------------------------------------
     -f !              

0

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** for the mutated structures.

The force field used in this tutorial is [**amber99sb-ildn-mut**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn), with **pmx library of modelled mutations** (mut). The path to the particular force field used is given as a property to the building block (**gmxlib**), and can be changed to the appropriate location.

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_md.gromacs.pdb2gmx**
***

In [5]:
# Create system topology

# Import module
from biobb_md.gromacs.pdb2gmx import Pdb2gmx

#### State A (WT->Mut) ####

# Create inputs/outputs
output_pdb2gmxA_gro = 'pdb2gmxA.pdb'
output_pdb2gmxA_top_zip = 'pdb2gmxA_top.zip'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'gmxlib' : gmxlib
}

# Create and launch bb
Pdb2gmx(input_pdb_path=output_structure_mutA,
        output_gro_path=output_pdb2gmxA_gro,
        output_top_zip_path=output_pdb2gmxA_top_zip,
        properties=prop).launch()

#### State B (Mut->WT) ####

# Create inputs/outputs
output_pdb2gmxB_gro = 'pdb2gmxB.pdb'
output_pdb2gmxB_top_zip = 'pdb2gmxB_top.zip'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'gmxlib' : gmxlib
}

# Create and launch bb
Pdb2gmx(input_pdb_path=output_structure_mutB,
        output_gro_path=output_pdb2gmxB_gro,
        output_top_zip_path=output_pdb2gmxB_top_zip,
        properties=prop).launch()

2019-12-04 17:45:10,669 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2019-12-04 17:45:10,672 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:45:13,488 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f mutA.pdb -o pdb2gmxA.pdb -p p2g.top -water spce -ff amber99sb-star-ildn-mut -i posre.itp

2019-12-04 17:45:13,489 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:45:13,490 [MainThread  ] [INFO ]  
Using the Amber99sb-star-ildn-mut force field in directory /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/mutff45/amber99sb-star-ildn-mut.ff

going to rename /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/mutff45/amber99sb-star-ildn-mut.ff/aminoacids.r2b
going to rename /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/mutff45/amber99sb-star-ildn-mut.ff/dna.

2019-12-04 17:45:13,492 [MainThread  ] [INFO ]  Compressing topology to: pdb2gmxA_top.zip
2019-12-04 17:45:13,494 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/forcefield.itp
2019-12-04 17:45:13,518 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/spce.itp
2019-12-04 17:45:13,520 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/ions.itp
2019-12-04 17:45:13,530 [MainThread  ] [INFO ]  Adding:
2019-12-04 17:45:13,534 [MainThread  ] [INFO ]  ['p2g.top', 'p2g_Ion2.itp', 'p2g_Protein.itp', 'posre_Ion2.itp', 'posre_Protein.itp']
2019-12-04 17:45:13,535 [MainThread  ] [INFO ]  to: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pdb2gmxA_top.zip
2019-12-04 17:45:13,536 [MainThread  ] [INFO ]  Removed: ['p2g.top']
2019-12-04 17:45:13,556 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2019-12-04 17:45:13,558 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:45:16,938 [M

2019-12-04 17:45:16,942 [MainThread  ] [INFO ]  Compressing topology to: pdb2gmxB_top.zip
2019-12-04 17:45:16,943 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/forcefield.itp
2019-12-04 17:45:16,958 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/spce.itp
2019-12-04 17:45:16,959 [MainThread  ] [INFO ]  Ignored file amber99sb-star-ildn-mut.ff/ions.itp
2019-12-04 17:45:16,963 [MainThread  ] [INFO ]  Adding:
2019-12-04 17:45:16,964 [MainThread  ] [INFO ]  ['p2g.top', 'p2g_Ion2.itp', 'p2g_Protein.itp', 'posre_Ion2.itp', 'posre_Protein.itp']
2019-12-04 17:45:16,967 [MainThread  ] [INFO ]  to: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pdb2gmxB_top.zip
2019-12-04 17:45:16,970 [MainThread  ] [INFO ]  Removed: ['p2g.top']


0

<a id="hybridtop"></a>
## Generate Hybrid Topology
**Generate Hybrid Topology** for the mutated structure using **pmx** package, adding the **morphing parameters**. 

*Note:* This tutorial is using a **Docker container** to run **pmx**. To use a local installation of **pmx**, just modify the appropriate properties parameter **pmx_cli_path**. 
***
**Building Blocks** used:
 - [Gentop](https://biobb-pmx.readthedocs.io/en/latest/pmx.html#module-pmx.gentop) from **biobb_pmx.pmx.gentop**
***

In [6]:
# pmx gentop: Gentop command (Generate Hybrid Topology) from pmx package

# Import module
from biobb_pmx.pmx.gentop import Gentop

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_pmxtopA_top_zip = 'pmxA_top.zip'
output_pmxtopA_log = 'pmxA_top.log'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'container_path' : 'docker',
    'container_image' : 'mmbirb/pmx'
    #'pmx_cli_path' : 'PATH/pmx-master/pmx/scripts/cli.py'
}

#Create and launch bb
Gentop(input_top_zip_path=output_pdb2gmxA_top_zip,
       output_top_zip_path=output_pmxtopA_top_zip,
       output_log_path=output_pmxtopA_log,
       properties=prop).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_pmxtopB_top_zip = 'pmxB_top.zip'
output_pmxtopB_log = 'pmxB_top.log'
  
prop = {
    'force_field' : 'amber99sb-star-ildn-mut',
    'container_path' : 'docker',
    'container_image' : 'mmbirb/pmx'
    #'pmx_cli_path' : 'PATH/pmx-master/pmx/scripts/cli.py'
}

# Create and launch bb
Gentop(input_top_zip_path=output_pdb2gmxB_top_zip,
       output_top_zip_path=output_pmxtopB_top_zip,
       output_log_path=output_pmxtopB_log,
       properties=prop).launch()

2019-12-04 17:45:16,988 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pdb2gmxA_top.zip
2019-12-04 17:45:16,989 [MainThread  ] [INFO ]  to:
2019-12-04 17:45:16,990 [MainThread  ] [INFO ]  ['260063cd-df72-4b74-90a9-8f19e6c9e4d6/p2g.top', '260063cd-df72-4b74-90a9-8f19e6c9e4d6/p2g_Ion2.itp', '260063cd-df72-4b74-90a9-8f19e6c9e4d6/p2g_Protein.itp', '260063cd-df72-4b74-90a9-8f19e6c9e4d6/posre_Ion2.itp', '260063cd-df72-4b74-90a9-8f19e6c9e4d6/posre_Protein.itp']
2019-12-04 17:45:16,990 [MainThread  ] [INFO ]  Unzipping: 
2019-12-04 17:45:16,991 [MainThread  ] [INFO ]  pdb2gmxA_top.zip
2019-12-04 17:45:16,992 [MainThread  ] [INFO ]  To: 
2019-12-04 17:45:16,993 [MainThread  ] [INFO ]  260063cd-df72-4b74-90a9-8f19e6c9e4d6/p2g.top
2019-12-04 17:45:16,994 [MainThread  ] [INFO ]  260063cd-df72-4b74-90a9-8f19e6c9e4d6/p2g_Ion2.itp
2019-12-04 17:45:16,994 [MainThread  ] [INFO ]  260063cd-df72-4b74-90a9-8f19e6c9e4d

2019-12-04 17:45:24,277 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:45:24,279 [MainThread  ] [INFO ]  ---------------------------------------------------------------------------------------------------------
 Program: gentop | pmx version 2.0+9.geb3590f
---------------------------------------------------------------------------------------------------------
     File Options          | Type(s)|Mode    | File(s)                | Description 
---------------------------------------------------------------------------------------------------------
     -p                    | top|r           | topol.top              | Input Topology File 
     -itp !                | itp|r           | /inout/p2g_Protein.itp   | Optional Input ITP  File 
     -o !                  | top,itp|w       | /inout/p2g_Protein.itp   | Topology or ITP output file  
     -ff !                 | ff|dir          | amber99sb-star-ildn-mut.ff   | Mutation force field  
     -log !                | log|w          

2019-12-04 17:45:25,696 [MainThread  ] [INFO ]  Using Docker image mmbirb/pmx
2019-12-04 17:45:27,050 [MainThread  ] [INFO ]  docker run -v /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/5099007e-3eb2-4039-b5e0-c83000bcca62:/inout mmbirb/pmx /bin/bash -c "cli.py gentop -o /inout/p2g.top -ff amber99sb-star-ildn-mut -log /inout/pmxB_top.log -p /inout/p2g.top"

2019-12-04 17:45:27,054 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:45:27,056 [MainThread  ] [INFO ]  ---------------------------------------------------------------------------------------------------------
 Program: gentop | pmx version 2.0+9.geb3590f
---------------------------------------------------------------------------------------------------------
     File Options          | Type(s)|Mode    | File(s)                | Description 
---------------------------------------------------------------------------------------------------------
     -p !                  |

0

<a id="index"></a>
## Creating an index file for the dummy atoms in the morphed structure

Some of the **mutations** done to the protein residues involve the generation of **dummy atoms**, atoms that are **slowly appearing** during the **transition** from the WT to the mutated structure. These **dummy atoms** need to be energy minimized before starting the **thermodynamic integration** step. If there are **no dummy atoms** in the corresponding state, this **energy minimization step can be omitted**.

In this particular example, the **WT to mutated protein transition** (Isoleucine to Alanine) is not generating any **dummy atoms**, so it does not need any minimization step. Conversely, the **mutated to WT transition** (Alanine to Isoleucine) is generating 9 **dummy atoms**, so the minimization step is needed for the **reverse transition** (stateB).

The **GROMACS** index file is built to **identify the dummy atoms** in the following **energy minimization** step, generating a new **FREEZE group** containing all atoms of the system except the **dummy atoms**. In the minimization process, this group will be **kept frozen**, whereas the **dummy atoms** will be left able to move. 
***
**Building Blocks** used:
 - [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_md.gromacs.make_ndx**
***

In [7]:
# Gromacs make_ndx: GROMACS Make index command from biobb_md package
# IMPORTANT: Only needed for stateB

# Import module
from biobb_md.gromacs.make_ndx import MakeNdx

# Create prop dict and inputs/outputs
output_ndx = 'indexB.ndx'
  
prop = {
    'selection' : 'a D*\n0 & ! 19\nname 20 FREEZE'
}

# Create and launch bb
MakeNdx(input_structure_path=output_pdb2gmxB_gro,
        output_ndx_path=output_ndx,
        properties=prop).launch()

2019-12-04 17:46:11,069 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2019-12-04 17:46:11,071 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:46:11,592 [MainThread  ] [INFO ]  echo 'a D*
0 & ! 19
name 20 FREEZE 
 q' | gmx -nobackup -nocopyright make_ndx -f pdb2gmxB.pdb -o indexB.ndx

2019-12-04 17:46:11,593 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:46:11,594 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:   135    Protein residues
There are:    75      Other residues
There are: 10949      Water residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 35147 atoms
  1 Protein             :  2207 atoms
  2 Protein-H           :  1084 atoms
  3 C-alpha             :   135 atoms
  4 Backbone            :   405 atoms
  5 MainChain           :   541 atoms
  6 MainChain+Cb        :   667 atoms
  7 MainChain+H         :   6

0

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **mutated protein** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **dummy atoms** till reaching a force of 100 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization

Method used to run the **energy minimization** is a **steepest descent**, with a **maximum force of 100 KJ/mol\*nm^2**, and a minimization **step size of 1fs**. The **maximum number of steps** to perform if the maximum force is not reached is **10,000 steps**. The previously generated **FREEZE** group is used to keep the system frozen except for the **dummy atoms**.

Please note that as previously described, for the stateA (forward transition, Isoleucine to Alanine mutation), as there are no dummies, **the energy minimization is omitted** and **the energy minimization step is skipped**. 

In [8]:
# Grompp: Creating portable binary run file for dummy atoms energy minimization
from biobb_md.gromacs.grompp import Grompp

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_tpr_min = 'em.tpr'

prop = {
    'gmxlib' : gmxlib,
    'mdp':{
        'type': 'minimization',
        'integrator' : 'steep',
        'emtol': '100',
        'dt': '0.001',
        'nsteps':'10000',
        'nstcomm': '1',
        'nstcalcenergy': '1',
        'freezegrps' : 'FREEZE',
        'freezedim' : "Y Y Y"
    }
}

# Create and launch bb
Grompp(input_gro_path=output_pdb2gmxB_gro,
       input_top_zip_path=output_pmxtopB_top_zip,
       input_ndx_path=output_ndx,
       output_tpr_path=output_tpr_min,
       properties=prop).launch()

2019-12-04 17:46:14,262 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-12-04 17:46:14,268 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmxB_top.zip
2019-12-04 17:46:14,270 [MainThread  ] [INFO ]  to:
2019-12-04 17:46:14,271 [MainThread  ] [INFO ]  ['8a77da46-20f2-406a-9496-7475f7450634/p2g.top', '8a77da46-20f2-406a-9496-7475f7450634/p2g_Ion2.itp', '8a77da46-20f2-406a-9496-7475f7450634/p2g_Protein.itp', '8a77da46-20f2-406a-9496-7475f7450634/posre_Ion2.itp']
2019-12-04 17:46:14,273 [MainThread  ] [INFO ]  Unzipping: 
2019-12-04 17:46:14,274 [MainThread  ] [INFO ]  pmxB_top.zip
2019-12-04 17:46:14,275 [MainThread  ] [INFO ]  To: 
2019-12-04 17:46:14,276 [MainThread  ] [INFO ]  8a77da46-20f2-406a-9496-7475f7450634/p2g.top
2019-12-04 17:46:14,278 [MainThread  ] [INFO ]  8a77da46-20f2-406a-9496-7475f7450634/p2g_Ion2.itp
2019-12-04 17:46:14,279 [MainThread  ] [INFO ]  8a77da46-20f2-4

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step.

In [9]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun

# Create prop dict and inputs/outputs
output_min_trr = 'emout.trr'
output_min_gro = 'emout.gro'
output_min_edr = 'emout.edr'
output_min_log = 'emout.log'

# Create and launch bb
Mdrun(input_tpr_path=output_tpr_min,
      output_trr_path=output_min_trr,
      output_gro_path=output_min_gro,
      output_edr_path=output_min_edr,
      output_log_path=output_min_log).launch()

2019-12-04 17:46:17,086 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-12-04 17:46:17,088 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:46:22,450 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s em.tpr -o emout.trr -c emout.gro -e emout.edr -g emout.log -nt 0

2019-12-04 17:46:22,451 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:46:22,451 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /anaconda3/envs/biobb_dev/bin/gmx
Data prefix:  /anaconda3/envs/biobb_dev
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s em.tpr -o emout.trr -c emout.gro -e emout.edr -g emout.log -nt 0

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This might affect your s

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [10]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import GMXEnergy

# Create prop dict and inputs/outputs
output_min_ene_xvg = 'min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
GMXEnergy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop).launch()

2019-12-04 17:46:25,887 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:46:25,900 [MainThread  ] [INFO ]  gmx energy -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/emout.edr -o min_ene.xvg -xvg none < 0967b207-d076-4fad-a425-8a2d1304b4bf/instructions.in

2019-12-04 17:46:25,901 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:46:25,902 [MainThread  ] [INFO ]  
Statistics over 85 steps [ 0.0000 through 84.0000 ps ], 1 data sets
All statistics are over 64 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -525710         41    163.546   -245.001  (kJ/mol)

2019-12-04 17:46:25,903 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman 

0

In [11]:
import plotly
import plotly.graph_objs as go

# Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
})

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://pmx.mpibpc.mpg.de/sardinia2018_tutorial1/eq.mdp) the simulated time was 100ps.*

In [12]:
# Grompp: Creating portable binary run file for system equilibration
from biobb_md.gromacs.grompp import Grompp

#### State A (WT->Mut) ####


cwd = os.getcwd()
gmxlib = cwd + "/pmx_tutorial/mutff45"

# Create prop dict and inputs/outputs
output_tprA_eq = 'eqA_20ps.tpr'

prop = {
    'gmxlib' : gmxlib,
    'mdp':{
        'type': 'free',
        'nsteps':'5000',
        'nstcomm' : '1',
        'nstcalcenergy' : '1'
    }
}

#Create and launch bb
Grompp(input_gro_path=output_pdb2gmxA_gro,
       input_top_zip_path=output_pmxtopA_top_zip,
       output_tpr_path=output_tprA_eq,
       properties=prop).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_tprB_eq = 'eqB_20ps.tpr'

prop = {
    'gmxlib' : gmxlib,
    'mdp':{
        'type': 'free',
        'nsteps':'5000',
        'nstcomm' : '1',
        'nstcalcenergy' : '1'
    }
}
#Create and launch bb
Grompp(input_gro_path=output_min_gro,
       input_top_zip_path=output_pmxtopB_top_zip,
       output_tpr_path=output_tprB_eq,
       properties=prop).launch()


2019-12-04 17:46:30,856 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-12-04 17:46:30,861 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmxA_top.zip
2019-12-04 17:46:30,861 [MainThread  ] [INFO ]  to:
2019-12-04 17:46:30,862 [MainThread  ] [INFO ]  ['6751fdee-56d9-470a-b29e-a7f5d0966c07/p2g.top', '6751fdee-56d9-470a-b29e-a7f5d0966c07/p2g_Ion2.itp', '6751fdee-56d9-470a-b29e-a7f5d0966c07/p2g_Protein.itp', '6751fdee-56d9-470a-b29e-a7f5d0966c07/posre_Ion2.itp']
2019-12-04 17:46:30,863 [MainThread  ] [INFO ]  Unzipping: 
2019-12-04 17:46:30,863 [MainThread  ] [INFO ]  pmxA_top.zip
2019-12-04 17:46:30,864 [MainThread  ] [INFO ]  To: 
2019-12-04 17:46:30,865 [MainThread  ] [INFO ]  6751fdee-56d9-470a-b29e-a7f5d0966c07/p2g.top
2019-12-04 17:46:30,865 [MainThread  ] [INFO ]  6751fdee-56d9-470a-b29e-a7f5d0966c07/p2g_Ion2.itp
2019-12-04 17:46:30,868 [MainThread  ] [INFO ]  6751fdee-56d9-4

2019-12-04 17:46:31,645 [MainThread  ] [INFO ]  Removed: ['mdout.mdp', 'e5340a80-ea8c-4a0a-bfe1-383d2dcca8e1', 'cb9807ee-73c1-4c7b-bc78-96b166ca9fd2']


0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [13]:
# Mdrun: Running equilibration
from biobb_md.gromacs.mdrun import Mdrun

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_eqA_trr = 'eqoutA.trr'
output_eqA_gro = 'eqoutA.gro'
output_eqA_edr = 'eqoutA.edr'
output_eqA_log = 'eqoutA.log'

# Create and launch bb
Mdrun(input_tpr_path=output_tprA_eq,
      output_trr_path=output_eqA_trr,
      output_gro_path=output_eqA_gro,
      output_edr_path=output_eqA_edr,
      output_log_path=output_eqA_log).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_eqB_trr = 'eqoutB.trr'
output_eqB_gro = 'eqoutB.gro'
output_eqB_edr = 'eqoutB.edr'
output_eqB_log = 'eqoutB.log'

# Create and launch bb
Mdrun(input_tpr_path=output_tprB_eq,
      output_trr_path=output_eqB_trr,
      output_gro_path=output_eqB_gro,
      output_edr_path=output_eqB_edr,
      output_log_path=output_eqB_log).launch()

2019-12-04 17:46:34,041 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-12-04 17:46:34,042 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:48:49,726 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s eqA_20ps.tpr -o eqoutA.trr -c eqoutA.gro -e eqoutA.edr -g eqoutA.log -nt 0

2019-12-04 17:48:49,727 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:48:49,728 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /anaconda3/envs/biobb_dev/bin/gmx
Data prefix:  /anaconda3/envs/biobb_dev
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s eqA_20ps.tpr -o eqoutA.trr -c eqoutA.gro -e eqoutA.edr -g eqoutA.log -nt 0

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [14]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import GMXEnergy

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_eqA_pd_xvg = 'eqA_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
GMXEnergy(input_energy_path=output_eqA_edr, 
          output_xvg_path=output_eqA_pd_xvg, 
          properties=prop).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_eqB_pd_xvg = 'eqB_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
GMXEnergy(input_energy_path=output_eqB_edr, 
          output_xvg_path=output_eqB_pd_xvg, 
          properties=prop).launch()

2019-12-04 17:51:29,330 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:51:29,365 [MainThread  ] [INFO ]  gmx energy -f /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/eqoutA.edr -o eqA_PD.xvg -xvg none < 0e95cff0-7941-4d3b-8ee8-ebe500d211b6/instructions.in

2019-12-04 17:51:29,371 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:51:29,372 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 2 data sets
All statistics are over 5001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -61.5082         --    1378.18   -76255.9  (bar)
Density                        1023         --    10.7769    8.99755  (kg/m^3)

2019-12-04 17:51:29,373 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is writte

2019-12-04 17:51:29,449 [MainThread  ] [INFO ]  Removed: [PurePosixPath('d5496bf7-8977-4844-b2ed-cd2a483b75aa')]


0

Please note the information shown by the next plots **increases with the time simulated**, and are included as a **reference for more complex calculations**.   

In [15]:
import plotly
from plotly import tools
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_eqA_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = tools.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

/anaconda3/envs/biobb_dev/lib/python3.6/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [16]:
import plotly
from plotly import tools
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_eqB_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = tools.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Energy Simulation
**Alchemical transition (thermodynamic integration)** free energy estimation approach is used with **GROMACS**: **free energy** is switched on, the **initial lambda** is chosen as zero, and **delta-lambda** (per MD step) is set such that at the end of the simulation **lambda** is at 1 (1 / nsteps). The **dhdl files** (dH/dl) written as a result will contain the **work values** required to perform these **transitions**.

- [Step 1](#mdStep1): Creating portable binary run file to run the **free energy simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free energy simulation

The **free_energy** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free energy simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 5000
-  free_energy              = yes
-  init_lambda              = 0
-  delta_lambda             = 0.0002
-  sc-alpha                 = 0
-  sc-sigma                 = 0.3

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, a total of **5,000 md steps** (10ps), all with the **free energy** flag turned on, with an **intial lambda** of 0 and a **delta lambda** of 0.00002.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://pmx.mpibpc.mpg.de/sardinia2018_tutorial1/eq.mdp) the simulated time was 100ps.*

In [17]:
# Grompp: Creating portable binary run file for thermodynamic integration (TI)
from biobb_md.gromacs.grompp import Grompp

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_tprA_ti = 'tiA.tpr'

prop = {
    'gmxlib' : gmxlib,
    'mdp':{
        'type': 'free',
        'nsteps':'5000',
        'free_energy' : 'yes',
        'init-lambda' : '0',
        'delta-lambda' : '2e-4',
        'sc-alpha' : '0.3',
        'sc-coul' : 'yes',
        'sc-sigma' : '0.25'
    }
}

# Create and launch bb
Grompp(input_gro_path=output_eqA_gro,
       input_top_zip_path=output_pmxtopA_top_zip,
       output_tpr_path=output_tprA_ti,
       properties=prop).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_tprB_ti = 'tiB.tpr'

prop = {
    'gmxlib' : gmxlib,
    'mdp':{
        'type': 'free',
        'nsteps':'5000',
        'free_energy' : 'yes',
        'init-lambda' : '0',
        'delta-lambda' : '4e-5',
        'sc-alpha' : '0.3',
        'sc-coul' : 'yes',
        'sc-sigma' : '0.25'
    }
}

# Create and launch bb
Grompp(input_gro_path=output_eqB_gro,
       input_top_zip_path=output_pmxtopB_top_zip,
       output_tpr_path=output_tprB_ti,
       properties=prop).launch()


2019-12-04 17:51:37,713 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2019-12-04 17:51:37,723 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmxA_top.zip
2019-12-04 17:51:37,724 [MainThread  ] [INFO ]  to:
2019-12-04 17:51:37,729 [MainThread  ] [INFO ]  ['b0f8adc2-8e6b-4da8-b308-e5571935f82b/p2g.top', 'b0f8adc2-8e6b-4da8-b308-e5571935f82b/p2g_Ion2.itp', 'b0f8adc2-8e6b-4da8-b308-e5571935f82b/p2g_Protein.itp', 'b0f8adc2-8e6b-4da8-b308-e5571935f82b/posre_Ion2.itp']
2019-12-04 17:51:37,734 [MainThread  ] [INFO ]  Unzipping: 
2019-12-04 17:51:37,740 [MainThread  ] [INFO ]  pmxA_top.zip
2019-12-04 17:51:37,744 [MainThread  ] [INFO ]  To: 
2019-12-04 17:51:37,746 [MainThread  ] [INFO ]  b0f8adc2-8e6b-4da8-b308-e5571935f82b/p2g.top
2019-12-04 17:51:37,750 [MainThread  ] [INFO ]  b0f8adc2-8e6b-4da8-b308-e5571935f82b/p2g_Ion2.itp
2019-12-04 17:51:37,754 [MainThread  ] [INFO ]  b0f8adc2-8e6b-4

2019-12-04 17:51:39,330 [MainThread  ] [INFO ]  Removed: ['mdout.mdp', '7385f7c7-8153-4820-af4c-e912a740e9b2', '56ec24b5-99f0-4008-bee0-25bf466752db']


0

<a id="mdStep2"></a>
### Step 2: Running free energy simulation

In [18]:
# Mdrun: Running equilibration
from biobb_md.gromacs.mdrun import Mdrun

#### State A (WT->Mut) ####

# Create prop dict and inputs/outputs
output_tiA_trr = 'tiA.trr'
output_tiA_gro = 'tiA.gro'
output_tiA_edr = 'tiA.edr'
output_tiA_log = 'tiA.log'
output_tiA_dhdl = 'tiA.dhdl'

# Create and launch bb
Mdrun(input_tpr_path=output_tprA_ti,
      output_trr_path=output_tiA_trr,
      output_gro_path=output_tiA_gro,
      output_edr_path=output_tiA_edr,
      output_log_path=output_tiA_log,
      output_dhdl_path=output_tiA_dhdl).launch()

#### State B (Mut->WT) ####

# Create prop dict and inputs/outputs
output_tiB_trr = 'tiB.trr'
output_tiB_gro = 'tiB.gro'
output_tiB_edr = 'tiB.edr'
output_tiB_log = 'tiB.log'
output_tiB_dhdl = 'tiB.dhdl'

# Create and launch bb
Mdrun(input_tpr_path=output_tprB_ti,
      output_trr_path=output_tiB_trr,
      output_gro_path=output_tiB_gro,
      output_edr_path=output_tiB_edr,
      output_log_path=output_tiB_log,
      output_dhdl_path=output_tiB_dhdl).launch()

2019-12-04 17:51:41,359 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2019-12-04 17:51:41,365 [MainThread  ] [INFO ]  Not using any container
2019-12-04 17:55:22,194 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s tiA.tpr -o tiA.trr -c tiA.gro -e tiA.edr -g tiA.log -nt 0 -dhdl tiA.dhdl

2019-12-04 17:55:22,196 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:55:22,197 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /anaconda3/envs/biobb_dev/bin/gmx
Data prefix:  /anaconda3/envs/biobb_dev
Working dir:  /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s tiA.tpr -o tiA.trr -c tiA.gro -e tiA.edr -g tiA.log -nt 0 -dhdl tiA.dhdl

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This mig

0

<a id="post"></a>
***
## Free Energy Estimation
The **Fast Growth TI** approach relies on **Jarzynski's equality** (when transition is performed in one direction only) or on the **Crooks Fluctuation Theorem** or the **Bennett Acceptance Ratio** (when the transitions are performed in both directions). 

**Workflow-generated results** should be used if a **minimum number of transitions** are calculated. In this particular case, as the tutorial is just computing 1 transition (forward + reverse), the number of work values computed are **not enough to extract the free energy**. Instead, we will use values taken from a **real run** of the snase example that can be found in the [pmx web page](http://pmx.mpibpc.mpg.de/sardinia2018_tutorial1/index.html).

- [Step 1](#tiStep1): Gathering together all the generated **dhdl files** (work values required to perform the transitions).
- [Step 2](#tiStep2): Compute the **free energy** using **Jarzynski's equality**, **Crooks Fluctuation Theorem** and  **Bennett Acceptance Ratio** with pmx.

*Note:* This tutorial is using a **Docker container** to run **pmx**. To use a local installation of **pmx**, just modify the appropriate properties parameter **pmx_cli_path**. 
***
**Building Blocks** used:
 - [Analyse](https://biobb-pmx.readthedocs.io/en/latest/pmx.html#module-pmx.analyse) from **biobb_pmx.pmx.analyse** 
***

<a id="tiStep1"></a>
### Step 1: Gathering together all the generated **dhdl files**
Gathering together all the generated **dhdl files** (work values required to perform the transitions). To be used as input for the final pmx free energy estimation.

In [19]:
# Gathering together all the generated dhdl files (work values required to perform the transitions) 
# from the free energy simulations.
# To be used as input for the final pmx free energy estimation.

#### State A (WT->Mut) ####

zf = zipfile.ZipFile('dhdlsA.zip', mode='w')

for file in os.listdir(os.getcwd()):
    if file.endswith("A.dhdl.xvg"):
            zf.write(file)
zf.close()

#### State B (Mut->WT) ####

zf = zipfile.ZipFile('dhdlsB.zip', mode='w')

for file in os.listdir(os.getcwd()):
    if file.endswith("B.dhdl.xvg"):
            zf.write(file)
zf.close()

<a id="tiStep2"></a>
### Step 2: Compute the free energy estimation
Compute the **free energy** using **Jarzynski's equality**, **Crooks Fluctuation Theorem** and **Bennett Acceptance Ratio** with pmx.

In [20]:
# pmx analyse: analyze_dhdl command from pmx package

# Import module
from biobb_pmx.pmx.analyse import Analyse

# Create prop dict and inputs/outputs

# Workflow-generated results should be used if a minimum number of transitions are calculated.
#state_A_xvg_zip = 'dhdlsA.zip'
#state_B_xvg_zip = 'dhdlsB.zip'

# In this particular case, as the tutorial is just computing 1 transition (forward + reverse),
# values taken from a real run of the snase example will be used instead.
state_A_xvg_zip = 'pmx_tutorial/dhdlA.zip'
state_B_xvg_zip = 'pmx_tutorial/dhdlB.zip'

output_result = 'pmx.outputs'
output_work_plot = 'pmx.plots.png'
  
prop = {
    'reverseB' : True,
    'container_path' : 'docker',
    'container_image' : 'mmbirb/pmx'
}

#Create and launch bb
Analyse(input_a_xvg_zip_path=state_A_xvg_zip,
        input_b_xvg_zip_path=state_B_xvg_zip,
        output_result_path=output_result,
        output_work_plot_path=output_work_plot,
        properties=prop).launch()


2019-12-04 17:59:14,376 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/pmx_tutorial/dhdlA.zip
2019-12-04 17:59:14,377 [MainThread  ] [INFO ]  to:
2019-12-04 17:59:14,377 [MainThread  ] [INFO ]  ['a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame0/dhdl0.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame1/dhdl1.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame10/dhdl10.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame11/dhdl11.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame12/dhdl12.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame13/dhdl13.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame14/dhdl14.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame15/dhdl15.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame16/dhdl16.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame17/dhdl17.xvg', 'a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame18/dhdl18.

2019-12-04 17:59:14,938 [MainThread  ] [INFO ]  Using Docker image mmbirb/pmx
2019-12-04 17:59:32,332 [MainThread  ] [INFO ]  docker run -v /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_pmx_tutorial/biobb_wf_pmx_tutorial/notebooks/62bea9a5-317d-4f80-8490-779edb548955:/inout mmbirb/pmx /bin/bash -c "cli.py analyse -fA /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame0/dhdl0.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame1/dhdl1.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame10/dhdl10.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame11/dhdl11.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame12/dhdl12.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame13/dhdl13.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame14/dhdl14.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame15/dhdl15.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame16/dhdl16.xvg /inout/a01ac477-274e-4343-901a-4fd8afde426f/st

2019-12-04 17:59:32,333 [MainThread  ] [INFO ]  Exit code 0

2019-12-04 17:59:32,334 [MainThread  ] [INFO ]  

                   PROCESSING THE DATA
  Forward Data
    # data points: 25001
    Length of trajectory:   50.000 ps
    Delta lambda:  0.00004
    Reading /inout/a01ac477-274e-4343-901a-4fd8afde426f/stateA/frame99/dhdl99.xvg

  Reverse Data
    # data points: 25001
    Length of trajectory:   50.000 ps
    Delta lambda: -0.00004
    Reading /inout/6141dcac-857b-4f9b-9450-fec7a73277d3/stateB/frame99/dhdl99.xvg

                       ANALYSIS
  Number of forward (0->1) trajectories: 100
  Number of reverse (1->0) trajectories: 100
  Temperature : 298.15 K

 --------------------------------------------------------
             Crooks Gaussian Intersection     
 --------------------------------------------------------
  Calculating Intersection...
  Bootstrap (Std Err): iteration 100/100
  CGI: Forward Gauss mean =    13.51 kJ/mol std =     4.03 kJ/mol
  CGI: Reverse Gauss mean 

PermissionError: [Errno 13] Permission denied: 'pmx.outputs'

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_result}}: **Final free energy estimation**. Summary of information got applying the different methods.
 - {{output_work_plot}}: **Final free energy plot** of the **Mutation free energy** pipeline.

***
<a id="questions"></a>
## Questions & Comments

Questions, issues, suggestions and comments are really welcome! 

* GitHub issues: 
    * https://github.com/bioexcel/biobb
* BioExcel forum:
    * https://ask.bioexcel.eu/  (TODO: biobb category)